In [1]:
from downloader import ENCODEDownloader
from pybedtools import BedTool
import pandas as pd
import gffutils
from processor import ENCODEProcessor

In [2]:
downloader = ENCODEDownloader()
# downloader.download_reference_genome()
# downloader.download_experiments(assay="ATAC-seq", n_experiments=2)

In [4]:
gene_db = downloader.get_gene_db(db_file="/home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite")

Database created


In [3]:
processor = ENCODEProcessor(output_dir="encode_data", db_file="/home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite")

Loading existing database from /home/tjkatz/genbio_hw/encode_data/genomes/GRCh38_v24_annotation.sqlite...
Database loaded successfully


In [4]:
t_to_g, g_to_t = processor.get_transcript_gene_mapping()

In [5]:
len(t_to_g), len(g_to_t)

(143984, 19844)

In [6]:
i = 0
for gene_id, transcript_ids in g_to_t.items():
    print(gene_id, transcript_ids)
    i += 1
    if i > 4:
        break

ENSG00000186092.4 ['ENST00000335137.3']
ENSG00000279928.1 ['ENST00000624431.1']
ENSG00000279457.3 ['ENST00000623834.3', 'ENST00000623083.3', 'ENST00000624735.1']
ENSG00000278566.1 ['ENST00000426406.2']
ENSG00000273547.1 ['ENST00000332831.3']


In [6]:
genes_bed = BedTool([
    (gene.seqid, gene.start - 1, gene.end, gene.id, 0, gene.strand)
    for gene in gene_db.features_of_type('gene')
])

In [8]:
atac_peaks = BedTool("/home/tjkatz/genbio_hw/encode_data/experiments/ATAC-seq/ENCSR504OUW/ENCFF578ODL.bed")
extended_peaks = atac_peaks.slop(b=10000, genome="hg38")
merged_peaks = extended_peaks.sort().merge()

In [9]:
peak_gene_intersect = merged_peaks.intersect(genes_bed, wo=True)

In [11]:
merged_peaks.head()

chr1	619075	644615
 chr1	768512	789132
 chr1	807200	880247
 chr1	894355	933959
 chr1	949102	1090370
 chr1	1105809	1261213
 chr1	1263576	1335119
 chr1	1339085	1360031
 chr1	1362123	1385703
 chr1	1389206	1451175
 

In [12]:
len(peak_gene_intersect)

41022

In [3]:
atac_peaks.head()

chr17	22521161	22521641	.	1000	.	43.16060	4190.77197	4183.08691	247	5.000000	5.000000	22521162	22521638	2048.82837	247	22521275	22521637	2112.65259	132
 chr3	93470286	93470867	.	1000	.	24.47231	3844.53003	3837.15942	242	5.000000	5.000000	93470286	93470865	1921.41821	245	93470286	93470865	2009.61560	239
 chr1	629075	630079	.	1000	.	4.82939	3321.11792	3314.13208	742	5.000000	5.000000	629080	630079	1688.07837	720	629078	630079	1818.21729	748
 chr1	633844	634276	.	1000	.	4.78651	3306.60278	3299.63013	166	5.000000	5.000000	633852	634276	1702.11597	159	633861	634272	1636.55872	148
 chr11	10509148	10509788	.	1000	.	25.85853	1957.33264	1950.85278	241	5.000000	5.000000	10509148	10509784	1010.71100	241	10509149	10509789	949.80609	240
 chr1	631857	632357	.	1000	.	3.70537	1916.37683	1909.92163	358	5.000000	5.000000	631857	632357	995.52039	360	631862	632356	946.83716	351
 chr4	165327252	165328013	.	1000	.	30.24539	1701.36377	1694.97412	267	5.000000	5.000000	165327252	165328012	837.13690	266	1653272

In [6]:
len(merged_peaks)

22551

In [7]:
merged_peaks.head()

chr1	619075	644615
 chr1	768512	789132
 chr1	807200	880247
 chr1	894355	933959
 chr1	949102	1090370
 chr1	1105809	1261213
 chr1	1263576	1335119
 chr1	1339085	1360031
 chr1	1362123	1385703
 chr1	1389206	1451175
 